# 巨量資料分析導論 Homework 5

# B036060012 政經大四 劉百耀

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("gender_submission.csv")

## 資料前處理

In [3]:
#看一下資料樣貌
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
#先將y label整理出來
Y_train = train["Survived"]

**先整理訓練資料集**

In [6]:
#看一下缺值情況
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


"Age"跟"Cabin"缺值較多，這裡不對此兩欄資料補值，因此不納入模型訓練;同時原資料的解答"Survived", 及"PassengerId", "Name", "Ticket"也將不納入訓練

In [7]:
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = train[selected_features]
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass      891 non-null int64
Sex         891 non-null object
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 41.8+ KB


可以發現"Embarked"還缺兩筆資料，因此以下需要對其補植，首先看一下"Embarked"內的資料分布

In [8]:
X_train.Embarked.describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

可以發現"S"出現頻率最高，因此以下直接將缺值處補上"S"

In [9]:
X_train.copy().Embarked[X_train.Embarked.isnull()] = "S"

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Pclass      891 non-null int64
Sex         891 non-null object
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 41.8+ KB


X_train部分資料已齊全，進行One-Hot Encoding

In [11]:
X_train = pd.get_dummies(X_train)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Pclass        891 non-null int64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Sex_female    891 non-null uint8
Sex_male      891 non-null uint8
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: float64(1), int64(3), uint8(5)
memory usage: 32.3 KB


**接著整理預測資料集**

In [13]:
#首先選擇與訓練資料集相同的特徵
X_test = test[selected_features]

In [14]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null object
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 19.7+ KB


可以發現"Fare"還缺一筆資料，以下需要對其補植，由於Fare為數值資料，因此以下直接給予平均值

In [15]:
X_test.iloc[X_test[X_test.Fare.isnull()].index, 4] = X_test.Fare.mean()

C:\Users\andy5\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null object
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        418 non-null float64
Embarked    418 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 19.7+ KB


X_test資料已齊全，進行One-Hot Encoding

In [17]:
X_test = pd.get_dummies(X_test)

In [18]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
Pclass        418 non-null int64
SibSp         418 non-null int64
Parch         418 non-null int64
Fare          418 non-null float64
Sex_female    418 non-null uint8
Sex_male      418 non-null uint8
Embarked_C    418 non-null uint8
Embarked_Q    418 non-null uint8
Embarked_S    418 non-null uint8
dtypes: float64(1), int64(3), uint8(5)
memory usage: 15.2 KB


## Neural Network模型訓練

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [20]:
model = Sequential()

model.add(Dense(units=80, input_dim=9, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=40, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=20, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=1, kernel_initializer='normal', activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
train_history = model.fit(X_train,
                          Y_train,
                          validation_split=0.1,
                          epochs=70,
                          batch_size=32,
                          verbose=1)

Train on 801 samples, validate on 90 samples
Epoch 1/70
801/801 [==============================] - 1s 714us/step - loss: 0.6922 - acc: 0.6167 - val_loss: 0.6900 - val_acc: 0.6222
Epoch 2/70
801/801 [==============================] - 0s 94us/step - loss: 0.6886 - acc: 0.6155 - val_loss: 0.6855 - val_acc: 0.6222
Epoch 3/70
801/801 [==============================] - 0s 111us/step - loss: 0.6840 - acc: 0.6155 - val_loss: 0.6765 - val_acc: 0.6222
Epoch 4/70
801/801 [==============================] - 0s 111us/step - loss: 0.6702 - acc: 0.6167 - val_loss: 0.6369 - val_acc: 0.6222
Epoch 5/70
801/801 [==============================] - 0s 114us/step - loss: 0.6187 - acc: 0.6292 - val_loss: 0.5446 - val_acc: 0.6444
Epoch 6/70
801/801 [==============================] - 0s 114us/step - loss: 0.5884 - acc: 0.6504 - val_loss: 0.5387 - val_acc: 0.7111
Epoch 7/70
801/801 [==============================] - 0s 99us/step - loss: 0.5684 - acc: 0.6617 - val_loss: 0.5229 - val_acc: 0.7333
Epoch 8/70
801/801 

801/801 [==============================] - 0s 95us/step - loss: 0.4793 - acc: 0.8002 - val_loss: 0.4049 - val_acc: 0.8111
Epoch 62/70
801/801 [==============================] - 0s 101us/step - loss: 0.4671 - acc: 0.8052 - val_loss: 0.4183 - val_acc: 0.8000
Epoch 63/70
801/801 [==============================] - 0s 116us/step - loss: 0.4703 - acc: 0.8040 - val_loss: 0.4024 - val_acc: 0.8333
Epoch 64/70
801/801 [==============================] - 0s 94us/step - loss: 0.4797 - acc: 0.8040 - val_loss: 0.3971 - val_acc: 0.8333
Epoch 65/70
801/801 [==============================] - 0s 116us/step - loss: 0.4925 - acc: 0.7953 - val_loss: 0.4115 - val_acc: 0.8000
Epoch 66/70
801/801 [==============================] - 0s 117us/step - loss: 0.4873 - acc: 0.8027 - val_loss: 0.4175 - val_acc: 0.8111
Epoch 67/70
801/801 [==============================] - 0s 94us/step - loss: 0.4761 - acc: 0.7978 - val_loss: 0.4068 - val_acc: 0.8222
Epoch 68/70
801/801 [==============================] - 0s 96us/step - 

## 預測

In [22]:
submission = pd.DataFrame({"PassengerId" : test.PassengerId.values, 
                           "Survived" : list(map(lambda x : 0 if x < 0.5 else 1, model.predict(X_test)))})


submission.to_csv("submission.csv", index = False)

## 預測結果

![img](capture.png)
上傳後準確率為76%，比原先退步。